<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2025_01_05_Query_Model_Cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to query the available LLMs, embedding models and reranking models via Graphlit API.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [3]:
from typing import List, Optional
from graphlit_api import QueryModelsModelsResults

async def query_models():
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_models()

        return response.models.results if response.models is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None


In [20]:
def pretty_print_model(card: QueryModelsModelsResults) -> str:
    # Safely unpack features lists (fall back to empty list if None)
    features = card.features
    metadata = card.metadata

    key_features = features.key_features if features is not None and features.key_features is not None else []
    strengths = features.strengths if features is not None and features.strengths is not None else []
    potential_use_cases = features.use_cases if features is not None and features.use_cases is not None else []

    # Determine maximum rows needed for the Features table
    max_feature_rows = max(len(key_features), len(strengths), len(potential_use_cases))

    # Build the Features table row by row
    feature_rows = []
    for i in range(max_feature_rows):
        key_col = f"- {key_features[i]}" if i < len(key_features) else ""
        strengths_col = f"- {strengths[i]}" if i < len(strengths) else ""
        potential_col = f"- {potential_use_cases[i]}" if i < len(potential_use_cases) else ""
        feature_rows.append(f"| {key_col} | {strengths_col} | {potential_col} |")

    if len(feature_rows) > 0:
        # Join all rows into a single string with the header
        features_table = (
            "| **Key Features**          | **Strengths**              | **Potential Use Cases**       |\n"
            "|---------------------------|----------------------------|--------------------------------|\n"
            + "\n".join(feature_rows)
        )
    else:
        features_table = "n/a"

    # Prepare data for the Metadata table
    metadata_table_rows = [
        f"| **Multilingual**            | {'Yes' if metadata and metadata.multilingual else 'No'} |",
        f"| **Multimodal**              | {'Yes' if metadata and metadata.multimodal else 'No'} |",
    ]

    if metadata is not None:
        if metadata.knowledge_cutoff:
            metadata_table_rows.append(f"| **Knowledge Cutoff**        | {metadata.knowledge_cutoff} |")
        if metadata.prompt_cost_per_million:
            metadata_table_rows.append(f"| **Prompt Cost per Million** | `${metadata.prompt_cost_per_million:,.2f}` |")
        if metadata.completion_cost_per_million:
            metadata_table_rows.append(f"| **Completion Cost per Million** | `${metadata.completion_cost_per_million:,.2f}` |")
        if metadata.embeddings_cost_per_million:
            metadata_table_rows.append(f"| **Embedding Cost per Million** | `${metadata.embeddings_cost_per_million:,.2f}` |")
        if metadata.reranking_cost_per_million:
            metadata_table_rows.append(f"| **Reranking Cost per Million** | `${metadata.reranking_cost_per_million:,.2f}` |")
        if metadata.context_window_tokens is not None:
            metadata_table_rows.append(f"| **Context Window Tokens**   | {metadata.context_window_tokens:,} |")
        if metadata.max_output_tokens is not None:
            metadata_table_rows.append(f"| **Max Output Tokens**       | {metadata.max_output_tokens:,} |")

    metadata_table = (
        "| **Property**               | **Value**                    |\n"
        "|----------------------------|------------------------------|\n"
        + "\n".join(metadata_table_rows)
    )

    # Construct the final Markdown
    markdown = f"""# {card.name}

### Model Type: {card.type}
**Model:** {card.model_type}.{card.model}

**Description:** {card.description}

**Model Card URI:** {card.uri}

**Available On:** {', '.join(card.available_on) if card.available_on else "N/A"}

---

## Features
{features_table}

---

## Metadata
{metadata_table}

---

> **Note**: All costs are in USD. Token limits and costs may vary depending on platform and configuration.
"""

    return markdown.strip()


Execute Graphlit example

In [21]:
from IPython.display import display, Markdown

models = await query_models()

if models is not None:
    for model in models:
        if model is not None:
            #print(pretty_print_model(model))
            display(Markdown(pretty_print_model(model)))

            print("---")
            print()

# jina-clip-v2

### Model Type: MULTIMODAL_EMBEDDING
**Model:** JinaModels.CLIP_Image

**Description:** The jina-clip-v2 model is a state-of-the-art CLIP-style model designed to handle both text and image data, offering multilingual support for 89 languages. It excels in high-resolution image processing with a resolution of 512x512 and employs Matryoshka representation learning for efficient truncated embeddings. This model is ideal for applications requiring robust multimodal capabilities, such as search and retrieval tasks across diverse languages and formats.

**Model Card URI:** https://huggingface.co/jinaai/jina-clip-v2

**Available On:** AWS SageMaker, Microsoft Azure, Google Cloud

---

## Features
| **Key Features**          | **Strengths**              | **Potential Use Cases**       |
|---------------------------|----------------------------|--------------------------------|
| - Multilingual support for 89 languages | - Handles both text and image data | - Search and retrieval tasks |
| - High image resolution at 512x512 | - Supports high-resolution image processing | - Multimodal applications |
| - Matryoshka representation learning | - Efficient truncated embeddings | - Cross-lingual tasks |

---

## Metadata
| **Property**               | **Value**                    |
|----------------------------|------------------------------|
| **Multilingual**            | Yes |
| **Multimodal**              | Yes |
| **Context Window Tokens**   | 8,191 |

---

> **Note**: All costs are in USD. Token limits and costs may vary depending on platform and configuration.

---



# jina-embeddings-v3

### Model Type: TEXT_EMBEDDING
**Model:** JinaModels.Embed_3_0

**Description:** The jina-embeddings-v3 model is a cutting-edge multilingual text embedding solution, boasting 570 million parameters and an impressive 8192 token-length capacity. It surpasses leading proprietary models from OpenAI and Cohere in performance on the MTEB benchmark, making it an excellent choice for applications requiring extensive text analysis and high accuracy in multilingual contexts.

**Model Card URI:** https://huggingface.co/jinaai/jina-embeddings-v3

**Available On:** AWS SageMaker, Microsoft Azure, Google Cloud

---

## Features
| **Key Features**          | **Strengths**              | **Potential Use Cases**       |
|---------------------------|----------------------------|--------------------------------|
| - 570M parameters | - High performance in multilingual contexts | - Multilingual text analysis |
| - 8192 token-length capacity | - Extensive text analysis capabilities | - High-accuracy applications |
| - Outperforms OpenAI and Cohere on MTEB |  | - Large-scale text processing |

---

## Metadata
| **Property**               | **Value**                    |
|----------------------------|------------------------------|
| **Multilingual**            | Yes |
| **Multimodal**              | No |
| **Context Window Tokens**   | 8,191 |

---

> **Note**: All costs are in USD. Token limits and costs may vary depending on platform and configuration.

---



# voyage-3-large

### Model Type: TEXT_EMBEDDING
**Model:** VoyageModels.Voyage_3_0_Large

**Description:** The Voyage-3-Large model is a robust text embedding solution designed to handle large-scale text data efficiently. It offers a generous free tier of 200 million tokens, making it an economical choice for businesses looking to leverage text embeddings without incurring high costs. With a competitive pricing structure of $0.18 per million tokens, it is ideal for applications requiring extensive text processing capabilities.

**Model Card URI:** None

**Available On:** N/A

---

## Features
n/a

---

## Metadata
| **Property**               | **Value**                    |
|----------------------------|------------------------------|
| **Multilingual**            | No |
| **Multimodal**              | No |
| **Context Window Tokens**   | 32,000 |

---

> **Note**: All costs are in USD. Token limits and costs may vary depending on platform and configuration.

...

---



# deepseek-chat

### Model Type: COMPLETION
**Model:** DeepseekModels.Chat

**Description:** We present DeepSeek-V3, a strong Mixture-of-Experts (MoE) language model with 671B total parameters with 37B activated for each token. To achieve efficient inference and cost-effective training, DeepSeek-V3 adopts Multi-head Latent Attention (MLA) and DeepSeekMoE architectures, which were thoroughly validated in DeepSeek-V2.

**Model Card URI:** https://huggingface.co/deepseek-ai/DeepSeek-V3

**Available On:** N/A

---

## Features
n/a

---

## Metadata
| **Property**               | **Value**                    |
|----------------------------|------------------------------|
| **Multilingual**            | No |
| **Multimodal**              | No |
| **Prompt Cost per Million** | `$0.27` |
| **Completion Cost per Million** | `$1.10` |
| **Context Window Tokens**   | 65,536 |
| **Max Output Tokens**       | 8,192 |

---

> **Note**: All costs are in USD. Token limits and costs may vary depending on platform and configuration.

---



# deepseek-coder

### Model Type: COMPLETION
**Model:** DeepseekModels.Coder

**Description:** We present DeepSeek-Coder-V2, an open-source Mixture-of-Experts (MoE) code language model that achieves performance comparable to GPT4-Turbo in code-specific tasks. Specifically, DeepSeek-Coder-V2 is further pre-trained from an intermediate checkpoint of DeepSeek-V2 with additional 6 trillion tokens.

**Model Card URI:** https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Instruct

**Available On:** N/A

---

## Features
n/a

---

## Metadata
| **Property**               | **Value**                    |
|----------------------------|------------------------------|
| **Multilingual**            | No |
| **Multimodal**              | No |
| **Prompt Cost per Million** | `$0.05` |
| **Completion Cost per Million** | `$0.08` |
| **Context Window Tokens**   | 65,536 |
| **Max Output Tokens**       | 8,192 |

---

> **Note**: All costs are in USD. Token limits and costs may vary depending on platform and configuration.

---

